In [1]:
#manipulate paths
import os 

#Cv library
import cv2 as cv

#Plot utilities
import matplotlib as mpl
import matplotlib.pyplot as plt #visualizzazione graph

# Image management
from skimage import io # Input/output
from skimage import transform # Geometric transformations (e.g. resize)
from skimage import filters # Local processing
from skimage import color # Change of color space
import IPython.display as ipd

# Set visualization size for figures (notebook only)
plt.rcParams['figure.figsize'] = [15,5] # width, height

#Numpy
import numpy as np #manipolare dati numerici

#Keras for modelling
import keras
from keras.models import Sequential, Model
from keras.layers import Dropout, Dense
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.preprocessing import image as kimage
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam # - Works #metter tensorflow prima
from keras.applications.mobilenet_v2 import preprocess_input as preprocess_mobilenet

#extra
import time
import random

In [2]:
#FACCIO PROVE TEST CAMERA COLORI - GRIGIO
import numpy as np
import cv2
cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height
while(True):
    ret, frame = cap.read()
    #frame = cv2.flip(frame, -1) # Flip camera vertically
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('frame', frame)
    cv2.imshow('gray', gray)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
# Caricamento modello per il rilevamento di volti frontali
face_detector = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

# renominate files

In [3]:
#utilizzo le foto già ritagliata e divise per persona, rinomino tutte le foto con UserX.count e le metto tutte in unico dataset

In [ ]:
import os
path = '/Users/gisto/Desktop/Digital Signal and Image Management/progetto/TRAIN/3-Copia/'
files = os.listdir(path)
i = 1

for file in files:
    os.rename(path+file,path+str("User.3.")+str(i)+'.jpg')
    i = i+1

# Creo il modello

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
# Path for face image database
path = 'dataset'#ttue le foto assieme
recognizer = cv2.face.LBPHFaceRecognizer_create()#pip install opencv-contrib-python #LBPHFaceRecognizer
#recognizer=cv2.face.EigenFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");
# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids

faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

#salvo il modello creato

recognizer.write('trainer/trainer.yml') 

#numero facce trovate
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

# Testo Modello

In [6]:
import cv2
import numpy as np
import os 
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')#modello salavtao prima 
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);
font = cv2.FONT_HERSHEY_SIMPLEX
#iniciate id counter
id = 0
# names related to ids: example ==> Marcelo: id=1,  etc
names = ['None', 'Leonardo', 'Riccardo', 'Giacomo'] 


cam = cv2.VideoCapture(0)
cam.set(3, 640) 
cam.set(4, 480) 


minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
while True:
    ret, img =cam.read()
   
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#gray scale
    
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        
        #calcolo confidenza in percentuale
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
        cv2.putText(
                    img, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   )  
    
    cv2.imshow('camera',img) 
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

        
cam.release()
cv2.destroyAllWindows()